In [22]:
from Crypto.Random import random

def randbytes(k):
    return random.getrandbits(8*k).to_bytes(k, byteorder='big')

def padPKCS7(x, k):
    ch = k - (len(x) % k)
    return x + bytes([ch] * ch)

In [23]:
# Taken from https://github.com/sfstpala/SlowSHA
class SHA1 (object):
    _default_h0, _default_h1, _default_h2, _default_h3, _default_h4, = (
        0x67452301, 0xefcdab89, 0x98badcfe, 0x10325476, 0xc3d2e1f0)

    def __init__(self, message, h0 = _default_h0, h1 = _default_h1, h2 = _default_h2, h3 = _default_h3, h4 = _default_h4, length = None):
        self._h0 = h0
        self._h1 = h1
        self._h2 = h2
        self._h3 = h3
        self._h4 = h4
        if length is None:
            length = len(message) * 8
        length = bin(length)[2:].rjust(64, "0")
        while len(message) > 64:
            self._handle(''.join(bin(i)[2:].rjust(8, "0")
                for i in message[:64]))
            message = message[64:]
        message = ''.join(bin(i)[2:].rjust(8, "0") for i in message) + "1"
        message += "0" * ((448 - len(message) % 512) % 512) + length
        for i in range(len(message) // 512):
            self._handle(message[i * 512:i * 512 + 512])


    def _handle(self, chunk):

        lrot = lambda x, n: (x << n) | (x >> (32 - n))
        w = []

        for j in range(len(chunk) // 32):
            w.append(int(chunk[j * 32:j * 32 + 32], 2))

        for i in range(16, 80):
            w.append(lrot(w[i - 3] ^ w[i - 8] ^ w[i - 14] ^ w[i - 16], 1)
                & 0xffffffff)

        a = self._h0
        b = self._h1
        c = self._h2
        d = self._h3
        e = self._h4

        for i in range(80):

            if i <= i <= 19:
                f, k = d ^ (b & (c ^ d)), 0x5a827999
            elif 20 <= i <= 39:
                f, k = b ^ c ^ d, 0x6ed9eba1
            elif 40 <= i <= 59:
                f, k = (b & c) | (d & (b | c)), 0x8f1bbcdc
            elif 60 <= i <= 79:
                f, k = b ^ c ^ d, 0xca62c1d6

            temp = lrot(a, 5) + f + e + k + w[i] & 0xffffffff
            a, b, c, d, e = temp, a, lrot(b, 30), c, d

        self._h0 = (self._h0 + a) & 0xffffffff
        self._h1 = (self._h1 + b) & 0xffffffff
        self._h2 = (self._h2 + c) & 0xffffffff
        self._h3 = (self._h3 + d) & 0xffffffff
        self._h4 = (self._h4 + e) & 0xffffffff

    def _digest(self):
        return (self._h0, self._h1, self._h2, self._h3, self._h4)

    def hexdigest(self):
        return ''.join(hex(i)[2:].rjust(8, "0")
            for i in self._digest())

    def digest(self):
        hexdigest = self.hexdigest()
        return bytes(int(hexdigest[i * 2:i * 2 + 2], 16)
            for i in range(len(hexdigest) // 2))

def authSHA1(key, message):
    return SHA1(key + message).digest()

In [41]:
from Crypto.Random import random
import struct

def padSHA1(s):
    l = len(s) * 8
    s += b'\x80'
    s += b'\x00' * ((56 - (len(s) % 64)) % 64)
    s += struct.pack('>Q', l)
    return s

keylen = random.randint(0, 100)
key = randbytes(keylen)

def validate(message, digest):
    return authSHA1(key, message) == digest

message = b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon'
messageDigest = authSHA1(key, message)

def forgeHash(keylen, message, digest, suffix):
    paddedForgedMessageWithKey = padSHA1(key + message) + suffix
    forgedMessage = paddedForgedMessageWithKey[keylen:]
    h = struct.unpack('>5I', digest)
    print (digest,h)
    forgedDigest = SHA1(suffix, h[0], h[1], h[2], h[3], h[4], len(paddedForgedMessageWithKey) * 8).digest()
    return (forgedMessage, forgedDigest)

def forgeValidatingHash(maxkeylen, message, digest, suffix):
    for i in range(maxkeylen):
        (forgedMessage, forgedDigest) = forgeHash(i, message, digest, suffix)
        if validate(forgedMessage, forgedDigest):
            return(forgedMessage, forgedDigest)
    raise Exception('unexpected')

print(forgeValidatingHash(100, message, messageDigest, b';admin=true'))

b'e\xd2\xa4\xc4\xe8\x17\xbf`|]\xe0\tA\x87D\x19\xacN\xcc\x18' (1708303556, 3893870432, 2086526985, 1099383833, 2890845208)
b'e\xd2\xa4\xc4\xe8\x17\xbf`|]\xe0\tA\x87D\x19\xacN\xcc\x18' (1708303556, 3893870432, 2086526985, 1099383833, 2890845208)
b'e\xd2\xa4\xc4\xe8\x17\xbf`|]\xe0\tA\x87D\x19\xacN\xcc\x18' (1708303556, 3893870432, 2086526985, 1099383833, 2890845208)
b'e\xd2\xa4\xc4\xe8\x17\xbf`|]\xe0\tA\x87D\x19\xacN\xcc\x18' (1708303556, 3893870432, 2086526985, 1099383833, 2890845208)
b'e\xd2\xa4\xc4\xe8\x17\xbf`|]\xe0\tA\x87D\x19\xacN\xcc\x18' (1708303556, 3893870432, 2086526985, 1099383833, 2890845208)
b'e\xd2\xa4\xc4\xe8\x17\xbf`|]\xe0\tA\x87D\x19\xacN\xcc\x18' (1708303556, 3893870432, 2086526985, 1099383833, 2890845208)
b'e\xd2\xa4\xc4\xe8\x17\xbf`|]\xe0\tA\x87D\x19\xacN\xcc\x18' (1708303556, 3893870432, 2086526985, 1099383833, 2890845208)
(b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [43]:
SHA1("").digest() # https://en.wikipedia.org/wiki/SHA-1

AttributeError: 'bytes' object has no attribute 'encode'

In [46]:
bytes('da39a3ee5e6b4b0d3255bfef95601890afd80709', h)

TypeError: string argument without an encoding